Hi EVERYONE,

Episode scraper of top team + feature expansion - distances etc
Feature expansion code can be used by EVERYONE as supplement to your mini maps or 115v2 as it speeds up any kind of training no matter how advanced your network is.

More work can be done to improve that expansions.
Generate vector line between ball and each player and between current player and each player, shortest distance from side lines and end lines and goal posts etc.
Based on those vectors you can calculate min and max distnace of each player to those lines.

In [ ]:
import os
import glob

files = glob.glob(r'/output/kaggle/working/*')
for items in files:
    os.remove(items)

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time

In [ ]:
MIN_FINAL_RATING = 1450 # top submission in a match must have reached this score
num_api_calls_today = 0
NUM_TEAMS = 1
EPISODES = 50

In [ ]:
BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

In [ ]:
# inital team list

r = requests.post(list_url, json = {"teamId":5653767}) # arbitrary ID, change to leading ID during challenge
rj = r.json()
teams_df = pd.DataFrame(rj['result']['teams'])

In [ ]:
teams_df.sort_values('publicLeaderboardRank', inplace = True)
# Selecting the top 10
teams_df = teams_df.iloc[:1,:]
teams_df

In [ ]:
def getTeamEpisodes(team_id,teams_df):

    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    # global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])

    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
    else:
        print('teams dataframe did not match')

    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;

    agents = []
    for i in range(len(team_episodes)):
        agents.append(team_episodes['agents'].loc[i])
    not_none_agents = []
    for i in range(len(agents)):
        l = [a['updatedScore'] for a in agents[i] if a['updatedScore'] is not None]
        if l:
            not_none_agents.append(agents[i])
            
    for i in range(len(not_none_agents)):
        agents = not_none_agents[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = max([a['submissionId'] for a in agents])
        team_episodes.loc[i, 'updatedScore'] = max([a['updatedScore'] for a in agents]) 
    
    team_episodes['final_score'] = team_episodes['updatedScore']
    team_episodes.sort_values('final_score', ascending = False, inplace=True)
    
    return rj, team_episodes

In [ ]:
def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})
        
    # save replay
    with open('{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])

In [ ]:
global num_api_calls_today

pulled_teams = {}
pulled_episodes = []
start_time = datetime.datetime.now()
r = BUFFER;

In [ ]:
while num_api_calls_today < EPISODES:
    # pull team
    top_teams = [i for i in teams_df.id if i not in pulled_teams]
    if len(top_teams) > 0:
        team_id = top_teams[0]
    else:
        break;
        
    # get team data
    team_json, team_df = getTeamEpisodes(team_id,teams_df); r+=1;
    
    num_api_calls_today+=1
    
    print('{} games for {}'.format(len(team_df), teams_df.loc[teams_df.id == team_id].iloc[0].teamName))

    
    team_df = team_df[  (MIN_FINAL_RATING is None or (team_df.final_score > MIN_FINAL_RATING))]
    
    print('   {} in score range from {} submissions'.format(len(team_df), len(team_df.submissionId.unique() ) ) )
    
    team_df = team_df[~team_df.id.isin(pulled_episodes)]        
    print('      {} remain to be downloaded\n'.format(len(team_df)))
        
    # pull games
    target_team_games = int(np.ceil(EPISODES / NUM_TEAMS))
    if target_team_games + len(pulled_episodes) > EPISODES:
        target_team_games = EPISODES - len(pulled_episodes)
     
    pulled_teams[team_id] = 0
    
    i = 0
    while i < len(team_df) and pulled_teams[team_id] < target_team_games:
        epid = team_df.id.iloc[i]
        if not (epid in pulled_episodes):
            try:
                saveEpisode(epid, team_json); r+=1;
                num_api_calls_today+=1
            except:
                time.sleep(20)
                i+=1;
                continue;
                
            pulled_episodes.append(epid)
            pulled_teams[team_id] += 1
            try:
                size = os.path.getsize('{}.json'.format(epid)) / 1e6
                print(str(num_api_calls_today) + ': Saved Episode #{} @ {:.1f}MB'.format(epid, size))
            except:
                print('  file {}.json did not seem to save'.format(epid))    
            if r > (datetime.datetime.now() - start_time).seconds:
                time.sleep( r - (datetime.datetime.now() - start_time).seconds)
                

        i+=1;
    print(); print()

In [ ]:
import os
import pandas as pd
a = ['WeKick']

json_paths = []
paths = ['/kaggle/working']
for path in paths:
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if len(filename) < 14:
                json_paths.append(os.path.join(dirname, filename))
                print('added')

In [ ]:
import os

print(os.getcwd())

In [ ]:
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def zip(): 
    # path to folder which needs to be zipped 
    directory = '/kaggle/working'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths:
        if file_name.endswith('.json'):
            print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('my_python_files.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths:
            if file.endswith('.json'):
                zip.write(file) 
  
    print('All files zipped successfully!')         
  
zip()

In [ ]:
def json_to_v115(json_path):
    raw_data = pd.read_json(json_path,
                        lines=True,
                        orient='columns')
    
    if ((raw_data['info'][0]["TeamNames"][0]) =='WeKick'):
            
        print('WeKick - correct')
            
    else:
        print('Return - skipping')
        return
    # Getting the final rewards of each player
    rewards = pd.DataFrame([_ for _ in raw_data.rewards])
    # Checking if there is an ERROR
    if rewards.iloc[0,0] is not None and rewards.iloc[0,1] is not None:
        # Getting steps features
        steps = pd.DataFrame([_ for _ in raw_data.steps])
        print(raw_data['info'][0]["TeamNames"][0])
        
        
        split_players = []
        for i in range(3002):
            temp = pd.DataFrame([_ for _ in steps[i]])
            split_players.append(temp)
        split_players = pd.concat(split_players)
        print(rewards.iloc[0,0])
        print(rewards.iloc[0,1])
        
        # Getting the winning player point of view
        if rewards.iloc[0,0] > rewards.iloc[0,1]:
            play1_feedback = split_players[0]  
        else:
            play1_feedback = split_players[1]
        print(play1_feedback.name)

        play1_feedback = play1_feedback.apply(pd.Series)
        
        # Getting player action
        play1_action = pd.DataFrame([_ for _ in play1_feedback.action])
        play1_action = play1_action.fillna(0)
        play1_action = play1_action.astype(int)
        play1_action = play1_action.rename(columns={play1_action.columns[0]: "Action"})
        
        # Getting player observations
        play1_obs = pd.DataFrame([_ for _ in play1_feedback.observation])

        play1_obs_raw = pd.DataFrame([_ for _ in play1_obs.players_raw])

        play1_obs_raw2 = pd.Series(play1_obs_raw[0], index=play1_obs_raw.index)
        obs_final = []
        for i,_ in play1_obs_raw2.items():
            temp = pd.DataFrame.from_dict(_,orient='index')
            obs_final.append(temp)
        obs_final = pd.concat(obs_final,axis=1)
        
        # Dropping unnecessary features

        obs_final = obs_final.drop(['designated', 'left_team_yellow_card','left_team_active','right_team_yellow_card','right_team_active','ball_owned_player',
                                'right_team_tired_factor','left_team_tired_factor','steps_left','score',
                                'right_team_roles','left_team_roles','sticky_actions','ball_rotation'], axis=0)

        obs_final = obs_final.transpose()
        
        # Extracting and adding necessary features

        ball = obs_final
        ball[['ball_x','ball_y','ball_z']] = pd.DataFrame(obs_final.ball.tolist(),index=obs_final.index)

        ball_dir = obs_final
        ball_dir[['ball_x_dir','ball_y_dir','ball_z_dir']] = pd.DataFrame(obs_final.ball_direction.tolist(),index=obs_final.index)

        left_coor = obs_final
        left_coor[['left1_coor','left2_coor','left3_coor','left4_coor','left5_coor','left6_coor','left7_coor','left8_coor','left9_coor',
                   'left10_coor','left11_coor']] = pd.DataFrame(obs_final.left_team.tolist(),index=obs_final.index)
        left_coor.head()

        left_coor[['left1_x','left1_y']] = pd.DataFrame(obs_final.left1_coor.tolist(),index=obs_final.index)
        left_coor[['left2_x','left2_y']] = pd.DataFrame(obs_final.left2_coor.tolist(),index=obs_final.index)
        left_coor[['left3_x','left3_y']] = pd.DataFrame(obs_final.left3_coor.tolist(),index=obs_final.index)
        left_coor[['left4_x','left4_y']] = pd.DataFrame(obs_final.left4_coor.tolist(),index=obs_final.index)
        left_coor[['left5_x','left5_y']] = pd.DataFrame(obs_final.left5_coor.tolist(),index=obs_final.index)
        left_coor[['left6_x','left6_y']] = pd.DataFrame(obs_final.left6_coor.tolist(),index=obs_final.index)
        left_coor[['left7_x','left7_y']] = pd.DataFrame(obs_final.left7_coor.tolist(),index=obs_final.index)
        left_coor[['left8_x','left8_y']] = pd.DataFrame(obs_final.left8_coor.tolist(),index=obs_final.index)
        left_coor[['left9_x','left9_y']] = pd.DataFrame(obs_final.left9_coor.tolist(),index=obs_final.index)
        left_coor[['left10_x','left10_y']] = pd.DataFrame(obs_final.left10_coor.tolist(),index=obs_final.index)
        left_coor[['left11_x','left11_y']] = pd.DataFrame(obs_final.left11_coor.tolist(),index=obs_final.index)

        left_coor_dir = obs_final
        left_coor_dir[['left1_coor_dir','left2_coor_dir','left3_coor_dir','left4_coor_dir','left5_coor_dir','left6_coor_dir','left7_coor_dir','left8_coor_dir','left9_coor_dir',
                   'left10_coor_dir','left11_coor_dir']] = pd.DataFrame(obs_final.left_team_direction.tolist(),index=obs_final.index)


        left_coor_dir[['left1_x_dir','left1_y_dir']] = pd.DataFrame(obs_final.left1_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left2_x_dir','left2_y_dir']] = pd.DataFrame(obs_final.left2_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left3_x_dir','left3_y_dir']] = pd.DataFrame(obs_final.left3_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left4_x_dir','left4_y_dir']] = pd.DataFrame(obs_final.left4_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left5_x_dir','left5_y_dir']] = pd.DataFrame(obs_final.left5_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left6_x_dir','left6_y_dir']] = pd.DataFrame(obs_final.left6_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left7_x_dir','left7_y_dir']] = pd.DataFrame(obs_final.left7_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left8_x_dir','left8_y_dir']] = pd.DataFrame(obs_final.left8_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left9_x_dir','left9_y_dir']] = pd.DataFrame(obs_final.left9_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left10_x_dir','left10_y_dir']] = pd.DataFrame(obs_final.left10_coor_dir.tolist(),index=obs_final.index)
        left_coor_dir[['left11_x_dir','left11_y_dir']] = pd.DataFrame(obs_final.left11_coor_dir.tolist(),index=obs_final.index)

        right_coor = obs_final
        right_coor[['right1_coor','right2_coor','right3_coor','right4_coor','right5_coor','right6_coor','right7_coor','right8_coor','right9_coor',
                   'right10_coor','right11_coor']] = pd.DataFrame(obs_final.right_team.tolist(),index=obs_final.index)


        right_coor[['right1_x','right1_y']] = pd.DataFrame(obs_final.right1_coor.tolist(),index=obs_final.index)
        right_coor[['right2_x','right2_y']] = pd.DataFrame(obs_final.right2_coor.tolist(),index=obs_final.index)
        right_coor[['right3_x','right3_y']] = pd.DataFrame(obs_final.right3_coor.tolist(),index=obs_final.index)
        right_coor[['right4_x','right4_y']] = pd.DataFrame(obs_final.right4_coor.tolist(),index=obs_final.index)
        right_coor[['right5_x','right5_y']] = pd.DataFrame(obs_final.right5_coor.tolist(),index=obs_final.index)
        right_coor[['right6_x','right6_y']] = pd.DataFrame(obs_final.right6_coor.tolist(),index=obs_final.index)
        right_coor[['right7_x','right7_y']] = pd.DataFrame(obs_final.right7_coor.tolist(),index=obs_final.index)
        right_coor[['right8_x','right8_y']] = pd.DataFrame(obs_final.right8_coor.tolist(),index=obs_final.index)
        right_coor[['right9_x','right9_y']] = pd.DataFrame(obs_final.right9_coor.tolist(),index=obs_final.index)
        right_coor[['right10_x','right10_y']] = pd.DataFrame(obs_final.right10_coor.tolist(),index=obs_final.index)
        right_coor[['right11_x','right11_y']] = pd.DataFrame(obs_final.right11_coor.tolist(),index=obs_final.index)

        right_coor_dir = obs_final
        right_coor_dir[['right1_coor_dir','right2_coor_dir','right3_coor_dir','right4_coor_dir','right5_coor_dir','right6_coor_dir','right7_coor_dir','right8_coor_dir','right9_coor_dir',
                   'right10_coor_dir','right11_coor_dir']] = pd.DataFrame(obs_final.right_team_direction.tolist(),index=obs_final.index)


        right_coor_dir[['right1_x_dir','right1_y_dir']] = pd.DataFrame(obs_final.right1_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right2_x_dir','right2_y_dir']] = pd.DataFrame(obs_final.right2_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right3_x_dir','right3_y_dir']] = pd.DataFrame(obs_final.right3_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right4_x_dir','right4_y_dir']] = pd.DataFrame(obs_final.right4_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right5_x_dir','right5_y_dir']] = pd.DataFrame(obs_final.right5_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right6_x_dir','right6_y_dir']] = pd.DataFrame(obs_final.right6_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right7_x_dir','right7_y_dir']] = pd.DataFrame(obs_final.right7_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right8_x_dir','right8_y_dir']] = pd.DataFrame(obs_final.right8_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right9_x_dir','right9_y_dir']] = pd.DataFrame(obs_final.right9_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right10_x_dir','right10_y_dir']] = pd.DataFrame(obs_final.right10_coor_dir.tolist(),index=obs_final.index)
        right_coor_dir[['right11_x_dir','right11_y_dir']] = pd.DataFrame(obs_final.right11_coor_dir.tolist(),index=obs_final.index)
        
        # One-Hot Encoding 

        ball_owning_labels = ['left_team_owning_ball','none_owning_ball','right_team_owning_ball']
        obs_final['ball_owning_cat'] = pd.cut(obs_final['ball_owned_team'],3,labels=ball_owning_labels)
        ball_owning_oh = pd.get_dummies(obs_final['ball_owning_cat'])

        active_labels = ['player0','player1','player2','player3','player4','player5','player6','player7','player8','player9','player10']
        obs_final['active_cat'] = pd.cut(obs_final['active'],11,labels=active_labels)
        active_oh = pd.get_dummies(obs_final['active_cat'])

        game_mode_labels = ['Normal','KickOff','GoalKick','FreeKick','Corner','ThrowIn','Penalty']
        obs_final['game_mode_cat'] = pd.cut(obs_final['game_mode'],7,labels=game_mode_labels)
        game_mode_oh = pd.get_dummies(obs_final['game_mode_cat'])
        
        # Dropping unnecessary features

        right_coor_drop = ['right1_coor','right1_coor','right2_coor','right3_coor','right4_coor','right5_coor','right6_coor','right7_coor','right8_coor','right9_coor','right10_coor','right11_coor']
        right_coor_dir_drop = ['right1_coor_dir','right1_coor_dir','right2_coor_dir','right3_coor_dir','right4_coor_dir','right5_coor_dir','right6_coor_dir','right7_coor_dir','right8_coor_dir','right9_coor_dir','right10_coor_dir','right11_coor_dir']
        left_coor_drop = ['left1_coor','left1_coor','left2_coor','left3_coor','left4_coor','left5_coor','left6_coor','left7_coor','left8_coor','left9_coor','left10_coor','left11_coor']
        left_coor_dir_drop = ['left1_coor_dir','left1_coor_dir','left2_coor_dir','left3_coor_dir','left4_coor_dir','left5_coor_dir','left6_coor_dir','left7_coor_dir','left8_coor_dir','left9_coor_dir','left10_coor_dir','left11_coor_dir']
        others_drop = ['right_team','right_team_direction','left_team','left_team_direction','active_cat','ball','game_mode_cat','ball_owning_cat','ball_direction','ball_owned_team','active','game_mode']
        obs_final = obs_final.drop(right_coor_drop,axis=1)
        obs_final = obs_final.drop(right_coor_dir_drop,axis=1)
        obs_final = obs_final.drop(left_coor_drop,axis=1)
        obs_final = obs_final.drop(left_coor_dir_drop,axis=1)
        obs_final = obs_final.drop(others_drop,axis=1)

        obs_final = pd.concat([obs_final,ball_owning_oh,active_oh,game_mode_oh],axis=1)
        obs_final.index = range(3002)
        obs_final = obs_final.join(play1_action)
        cols = ['left1_x','left1_y','left2_x','left2_y','left3_x','left3_y','left4_x','left4_y','left5_x','left5_y','left6_x','left6_y','left7_x','left7_y','left8_x','left8_y','left9_x','left9_y','left10_x','left10_y','left11_x','left11_y', 
                'left1_x_dir','left1_y_dir','left2_x_dir','left2_y_dir','left3_x_dir','left3_y_dir','left4_x_dir','left4_y_dir','left5_x_dir','left5_y_dir','left6_x_dir','left6_y_dir','left7_x_dir','left7_y_dir','left8_x_dir','left8_y_dir','left9_x_dir','left9_y_dir','left10_x_dir','left10_y_dir','left11_x_dir','left11_y_dir',
                'right1_x','right1_y','right2_x','right2_y','right3_x','right3_y','right4_x','right4_y','right5_x','right5_y','right6_x','right6_y','right7_x','right7_y','right8_x','right8_y','right9_x','right9_y','right10_x','right10_y','right11_x','right11_y',
                'right1_x_dir','right1_y_dir','right2_x_dir','right2_y_dir','right3_x_dir','right3_y_dir','right4_x_dir','right4_y_dir','right5_x_dir','right5_y_dir','right6_x_dir','right6_y_dir',
                'right7_x_dir','right7_y_dir','right8_x_dir','right8_y_dir','right9_x_dir','right9_y_dir','right10_x_dir','right10_y_dir','right11_x_dir','right11_y_dir', 
                'ball_x','ball_y','ball_z','ball_x_dir','ball_y_dir','ball_z_dir',
                'left_team_owning_ball','none_owning_ball','right_team_owning_ball',
                'player0','player1','player2','player3','player4','player5','player6','player7','player8','player9','player10',
                'Normal','KickOff','GoalKick','FreeKick','Corner','ThrowIn','Penalty','Action'
                ]
        obs_final = obs_final[cols]

        return obs_final
   
    else:
        return pd.DataFrame() 

In [ ]:
data = []
c = 0
for _ in json_paths:
    print(c)

    if c<EPISODES:
        pass
    else:
        break
    try:
        to_app = json_to_v115(_)
        if to_app.empty:
            print('NoneType')
        else:
            c = c+1
            print('count :', c)
            data.append(to_app)
    except:
        pass

data = pd.concat(data,axis=0)
data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [ ]:
df = data

In [ ]:
df.to_csv('115boosted.csv')

In [ ]:
def dst(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    dist = round(dist,3)
    return dist 

Calculates distances between ball and each player, distances between each player

In [ ]:
import math

for index in range(0, 44, 2 ):

#print('Column Number : ', index)
    colname = df.columns[index]
    colname2 = df.columns[index+1]
#print(len(df.columns))
    df['new_column_name'+str(index)] = df.apply(lambda x: dst(x['ball_x'], x['ball_y'],x[colname], x[colname2]), axis=1)

for index in range(2, 22, 2 ):

    #print('Column Number : ', index)
    colname = df.columns[index]
    colname2 = df.columns[index+1]
        #print(len(df.columns))
for index1 in range(24, 44, 2 ):
          # print('Column Number : ', index)
    colname3 = df.columns[index1]
    colname4 = df.columns[index1+1]
            #print(len(df.columns))
    df['new_column_name2'+str(index)+str(index1)] = df.apply(lambda x: dst(x[colname], x[colname2],x[colname3], x[colname4]), axis=1)

More work can be done to improve that expansions.
Generate vector line between ball and each player and between current player and each player, shortest distance from side lines and end lines and goal posts etc.
Based on those vectors you can calculate min and max distnace of each player to those lines.

In [ ]:
X = df.drop(['Action'],axis=1).values

y = df['Action'].values
y = y.astype(np.int32)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.02,random_state=42)

In [ ]:
clf = RandomForestClassifier()
model=clf.fit(X_train,y_train)
clf.score(X_test,y_test)

In [ ]:
# Install:
# Kaggle environments.
#!git clone https://github.com/Kaggle/kaggle-environments.git
#!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.6 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

!mkdir /kaggle_simulations
!mkdir /kaggle_simulations/agent
!mkdir /kaggle_simulations/agent/saved_model

In [ ]:
# save the model to disk
import pickle
filename = '/kaggle_simulations/agent/saved_model/model.sav'
pickle.dump(model, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)

In [ ]:
%%writefile /kaggle_simulations/agent/main.py
from kaggle_environments.envs.football.helpers import *
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle
import math

# load the model from disk
filename = '/kaggle_simulations/agent/saved_model/model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)

directions = [[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

#track raw data to troubleshoot...
track_raw_data=[]

perfectRange = [[0.7, 0.95], [-0.12, 0.12]]

def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def get_heading(pos1,pos2):
    raw_head=math.atan2(pos1[0]-pos2[0],pos1[1]-pos2[1])/math.pi*180

    if raw_head<0:
        head=360+raw_head
    else:
        head=raw_head
    return head

def get_action(action_num):
    if action_num==0:
        return Action.Idle
    if action_num==1:
        return Action.Left
    if action_num==2:
        return Action.TopLeft
    if action_num==3:
        return Action.Top
    if action_num==4:
        return Action.TopRight
    if action_num==5:
        return Action.Right
    if action_num==6:
        return Action.BottomRight
    if action_num==7:
        return Action.Bottom
    if action_num==8:
        return Action.BottomLeft
    if action_num==9:
        return Action.LongPass
    if action_num==10:
        return Action.HighPass
    if action_num==11:
        return Action.ShortPass
    if action_num==12:
        return Action.Shot
    if action_num==13:
        return Action.Sprint
    if action_num==14:
        return Action.ReleaseDirection
    if action_num==15:
        return Action.ReleaseSprint
    if action_num==16:
        #return Action.Sliding
        return Action.Idle
    if action_num==17:
        return Action.Dribble
    if action_num==18:
        #return Action.ReleaseDribble
        return Action.Idle
    return Action.Right

@human_readable_agent
def agent(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    x = controlled_player_pos[0]
    y = controlled_player_pos[1]
    pactive=obs['active']
    
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    
    # Make sure player is running.
    if  0 < controlled_player_pos[0] < 0.6 and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    elif 0.6 < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint
    
    #if we have the ball
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        dat=[]
        to_append=[]
        #return Action.Right
        #get controller player pos
        controlled_player_pos = obs['left_team'][obs['active']]
        
        if inside(controlled_player_pos, perfectRange) and controlled_player_pos[0] < obs['ball'][0]:
            return Action.Shot
        
        goalx=0.0
        goaly=0.0

        sidelinex=0.0
        sideliney=0.42
        
        goal_dist=get_distance((x,y),(goalx,goaly))
        sideline_dist=get_distance((x,y),(sidelinex,sideliney))
        to_append.append(goal_dist)
        to_append.append(sideline_dist)
        
        for i in range(len(obs['left_team'])):
            x=obs['left_team'][i][0]
            y=obs['left_team'][i][1]
            to_append.append(x)
            to_append.append(y)
            
        for i in range(len(obs['right_team'])):
            x=obs['right_team'][i][0]
            y=obs['right_team'][i][1]
            to_append.append(x)
            to_append.append(y)
        
        dat.append(to_append)
        
        predicted=loaded_model.predict(dat)
        
        do=get_action(predicted)
        
        if do == None:
            return Action.Right
        else:
            return do
    
    # if we don't have ball run to ball
    else:
        dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)
        #where ball is going
        ball_targetx=obs['ball'][0]+(obs['ball_direction'][0]*5)
        ball_targety=obs['ball'][1]+(obs['ball_direction'][1]*5)
        
        e_dist=get_distance(obs['left_team'][obs['active']],obs['ball'])
        
        if e_dist >.01:
            # Run where ball will be
            xdir = dirsign(ball_targetx - controlled_player_pos[0])
            ydir = dirsign(ball_targety - controlled_player_pos[1])
            return directions[ydir][xdir]
        else:
            # Run towards the ball.
            xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
            ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
            return directions[ydir][xdir]
        

In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle_simulations/agent/main.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

In [ ]:
!cd /kaggle_simulations/agent && tar -czvf /kaggle/working/submit.tar.gz main.py saved_model